# Inversion Intensity

Tracks left/right radiation points and X-point, including brightness (intensity) for D-IIID TV images

In [ ]:
import numpy as np
from scipy.io import readsav
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import cv2
from pathlib import Path

### Data Loading

In [ ]:
def _load_data(filename):
    dat = readsav(filename)
    emission = dat['emission_structure']
    return emission[0]

def _find_index(arr,val):
    return np.argmin(abs(arr-val))

In [ ]:
filepath = Path('tv_images')
filename = 'emission_structure_pu_cam240perp_189101.sav'
[inverted,radii,elevation,frames,times,vid_frames,vid_times,vid] = _load_data(filepath / filename)

### Static Image With Corner Detection

Inverted: R,Z Coordinate Array
Radii/Elevation are redundant across times. Can just use ones from t=0.

In [ ]:

fid = 50
tid = _find_index(vid_times,times[fid]) #find frame id for camera image with t=times[fid]
find_x = inverted[fid]
img = inverted[fid].copy()
gray=(255-255*(img-np.min(img))/(np.max(img)-np.min(img))).astype('uint8')
useHarrisDetector = False # False uses Shi-Tomasi Corner Detector
corners = np.intp(cv2.goodFeaturesToTrack(gray,3,.5,10, useHarrisDetector=useHarrisDetector))
x = radii[0][corners[:,0,0]]
y = elevation[0][corners[:,0,1]]
print(np.column_stack((x,y)))
plt.pcolormesh(radii[fid],elevation[fid],img,shading='auto', cmap='gray')
plt.scatter(x,y,color='red')
plt.title(f"Inverted with Radiation Points, Frame {fid}")
plt.show()

### Dynamic Point Tracking

In [ ]:
def get_avg_value(array):
    return np.sqrt(np.mean(np.square(array)))

# converts from natural units to indices
def convert_center(radii, elevation, value):
    rad_idx = (np.abs(radii - value[0])).argmin()
    elev_idx = (np.abs(elevation - value[1])).argmin()
    return rad_idx, elev_idx

# distance from test point to line created by point_1 and point_2
def get_dist_line(point_1, point_2, test_point):
    top = np.abs((point_2[0]-point_1[0])*(point_2[1]-test_point[1])-(point_1[0]-test_point[0])*(point_2[1]-point_1[1]))
    bottom = np.sqrt((point_2[0]-point_1[0])**2+(point_2[1]-point_1[1])**2)
    return top / bottom

In [ ]:
# boundary array for frame around center point
def get_bounds(centers, dist, im_size):
    
    bounds = np.array([centers - dist, centers + dist])
    
    bounds[bounds < 0] = 0 # set negative bounds to 0
    
    new_vals = bounds.copy()
    new_vals[new_vals > im_size[0]] = im_size[0]
    bounds[:,:,0] = new_vals[:,:,0] # set x bounds within x frame
    
    new_vals = bounds.copy()
    new_vals[new_vals > im_size[1]] = im_size[1]
    bounds[:,:,1] = new_vals[:,:,1] # set y bounds within y frame
    
    return bounds

# array of corners using Shi-Tomasi Corner Detector
def get_corners(img):
    
    gray = (255-255*(img-np.min(img))/(np.max(img)-np.min(img))).astype('uint8')
    corners = np.intp(cv2.goodFeaturesToTrack(gray,3,.5,10, useHarrisDetector=False))
    x = corners[:,0,0]
    y = corners[:,0,1]
    
    return np.column_stack((x,y))

# array of corner pixel frame intensities
def get_corner_values(img, corners, dist, im_size):
    
    bounds = get_bounds(corners, dist, im_size)
    
    avg_values = []
    
    for i in range(len(corners)):
        temp_frame = img[bounds[0,i,1]:bounds[1,i,1],bounds[0,i,0]:bounds[1,i,0]]
        avg_values.append(get_avg_value(temp_frame))
    
    return avg_values

Finding and merging the most likely radiation points
1. X-point is absolute
1. Merge radiation points that are close together
1. Provided that there also exists a corner that is close enough to the previous line formed by the x-point and radiation point, then replace radiation point with corner

In [ ]:
# merges points that are close together and sets new radiation points to possible corners if they meet threshold criteria
def merge_points(centers_old, centers_new, corners, avg_values, avg_values_corners, merge_threshold, distance_threshold):
    
    right_dist = []
    left_dist = []
    centers_new_update = centers_new.copy()
    avg_values_update = avg_values.copy()
    
    for i in range(len(corners)):
        right_dist.append(get_dist_line(centers_old[0],centers_old[1],corners[i]))
        left_dist.append(get_dist_line(centers_old[0],centers_old[2],corners[i]))

    l_x_dist = np.sqrt(np.sum(np.square(centers_new[0]-centers_new[1])))
    r_x_dist = np.sqrt(np.sum(np.square(centers_new[0]-centers_new[2])))
    
    if (l_x_dist < merge_threshold) and (np.min(left_dist) < distance_threshold):
        index = np.where(right_dist == np.min(left_dist))[0][0]
        centers_new_update[1] = corners[index]
        avg_values_update[1] = avg_values_corners[index]
    
    if (r_x_dist < merge_threshold) and (np.min(right_dist) < distance_threshold):
        index = np.where(right_dist == np.min(right_dist))[0][0]
        centers_new_update[2] = corners[index]
        avg_values_update[2] = avg_values_corners[index]
    
    return centers_new_update, avg_values_update

# array of values for x point and 2 radiation points
def get_center_values(img, centers, dist, im_size, merge_threshold, distance_threshold):
    
    bounds = get_bounds(centers, dist, im_size)
    
    x_frame = img[bounds[0,0,1]:bounds[1,0,1],bounds[0,0,0]:bounds[1,0,0]]
    l_frame = img[bounds[0,1,1]:bounds[1,1,1],bounds[0,1,0]:bounds[1,1,0]]
    r_frame = img[bounds[0,2,1]:bounds[1,2,1],bounds[0,2,0]:bounds[1,2,0]]
    
    local_x_max = np.unravel_index(np.argmax(x_frame), x_frame.shape)
    local_l_max = np.unravel_index(np.argmax(l_frame), l_frame.shape)
    local_r_max = np.unravel_index(np.argmax(r_frame), r_frame.shape)
    
    global_x_max = np.flip(np.ravel(local_x_max)) + [bounds[0,0,0], bounds[0,0,1]]
    global_l_max = np.flip(np.ravel(local_l_max)) + [bounds[0,1,0], bounds[0,1,1]]
    global_r_max = np.flip(np.ravel(local_r_max)) + [bounds[0,2,0], bounds[0,2,1]]
    
    centers_update = np.array([global_x_max, global_l_max, global_r_max])
    avg_values = np.array([get_avg_value(x_frame), get_avg_value(l_frame), get_avg_value(r_frame)])
    
    corners = get_corners(img)
    avg_values_corners = get_corner_values(img, corners, dist, im_size)
    new_centers, new_avg_vals = merge_points(centers, centers_update, corners, avg_values, avg_values_corners, merge_threshold, distance_threshold)
        
    return new_centers, new_avg_vals

# update frame with new centers if avg value is greater than threshold
def update_frame(img, centers, dist, im_size, intensity_threshold, merge_threshold, distance_threshold):
    
    new_centers, new_avg_vals = get_center_values(img, centers, dist, im_size, merge_threshold, distance_threshold)
    
    for i in range(3):
        if new_avg_vals[i] > intensity_threshold:
            centers[i] = new_centers[i]
    
    return centers, new_avg_vals

# main function
def main(input_array, centers_ini, dist, im_size, intensity_threshold, merge_threshold, distance_threshold, num_iter):
    centers = centers_ini.copy()
    centers_array = []
    avg_values_array = []
    
    for i in range(num_iter):
        img = input_array[i].copy()
        centers_update, avg_values_temp = update_frame(img, centers, dist, im_size, intensity_threshold, merge_threshold, distance_threshold)
        centers_array.append(centers_update)
        avg_values_array.append(avg_values_temp)
        centers = centers_update.copy()
    
    return np.array(centers_array), np.array(avg_values_array)

In [ ]:
# initialize values
x_0 = np.array([1.35,-1.05]) # x-point location
l_0 = np.array([1.0,-1.25]) # left radiation point location
r_0 = np.array([1.5,-1.25]) # right radiation point location

centers_0 = np.array([convert_center(radii,elevation,x_0),
                      convert_center(radii,elevation,l_0),
                      convert_center(radii,elevation,r_0)])

radius = 6 # for some reason, 6 just works the best while every other value doesn't work
threshold = 0.05 # intensity value threshold for points to update their position
merge_threshold = 20 # merge threshold for radiation point and X-point, should be on factor of 2X radius
distance_threshold = 10 # distance from emission line for corner to be considered a radiation point

# run main function
centers_array, avg_values_array = main(inverted, centers_0, radius, img.shape, threshold, merge_threshold, distance_threshold, len(inverted))

### Results

In [ ]:
# plot intensity values over each frame
plt.plot(avg_values_array)
plt.legend(['x','l','r'])
plt.title(filename)
plt.show()

Circle bounds around point is a bit misleading, it's actually a square. But differences shouldn't be too big.

Intensity is the tracked points. Corners is any corners that gets noticed by detector.

In [ ]:
# sample plot of calculated centers
idx = 1
x, y = zip(*centers_array[idx])
img = inverted[idx].copy()

scaling = radii[0,1]-radii[0,0]

fig, axs = plt.subplots()
circ_0 = plt.Circle((radii[0,x[0]], elevation[0,y[0]]), scaling*radius, color='tab:red', fill=False)
circ_1 = plt.Circle((radii[0,x[1]], elevation[0,y[1]]), scaling*radius, color='tab:red', fill=False)
circ_2 = plt.Circle((radii[0,x[2]], elevation[0,y[2]]), scaling*radius, color='tab:red', fill=False)
line1x = [radii[0,x[0]],radii[0,x[1]]]
line1y = [elevation[0,y[0]],elevation[0,y[1]]]
line2x = [radii[0,x[0]],radii[0,x[2]]]
line2y = [elevation[0,y[0]],elevation[0,y[2]]]

axs.pcolormesh(radii[0],elevation[0],img,shading='auto', cmap='gray')
axs.plot(line1x,line1y,color='tab:blue')
axs.plot(line2x,line2y,color='tab:blue')
axs.scatter(radii[0,x],elevation[0,y],s=5,c='tab:orange',label='intensity')
axs.add_artist(circ_0)
axs.add_artist(circ_1)
axs.add_artist(circ_2)

gray=(255-255*(img-np.min(img))/(np.max(img)-np.min(img))).astype('uint8')
corners = np.intp(cv2.goodFeaturesToTrack(gray,3,.5,10, useHarrisDetector=useHarrisDetector))
x1 = radii[idx][corners[:,0,0]]
y1 = elevation[idx][corners[:,0,1]]
axs.scatter(x1,y1,color='cyan',s=5,marker='x',label='corners')
plt.suptitle(filename)
plt.title(f'Time = {times[idx]-times[0]:.2f} ms')
plt.legend()
plt.show()

In [ ]:
# generate video
fig, axs = plt.subplots()

def animate(idx):
    x, y = zip(*centers_array[idx])
    img = inverted[idx].copy()

    circ_0 = plt.Circle((radii[0,x[0]], elevation[0,y[0]]), scaling*radius, color='tab:red', fill=False)
    circ_1 = plt.Circle((radii[0,x[1]], elevation[0,y[1]]), scaling*radius, color='tab:red', fill=False)
    circ_2 = plt.Circle((radii[0,x[2]], elevation[0,y[2]]), scaling*radius, color='tab:red', fill=False)
    
    line1x = [radii[0,x[0]],radii[0,x[1]]]
    line1y = [elevation[0,y[0]],elevation[0,y[1]]]
    line2x = [radii[0,x[0]],radii[0,x[2]]]
    line2y = [elevation[0,y[0]],elevation[0,y[2]]]

    axs.clear()
    
    axs.pcolormesh(radii[0],elevation[0],img,shading='auto', cmap='gray')
    axs.plot(line1x,line1y,color='tab:blue')
    axs.plot(line2x,line2y,color='tab:blue')
    axs.scatter(radii[0,x],elevation[0,y],s=5,c='tab:orange',label='intensity')
    axs.add_artist(circ_0)
    axs.add_artist(circ_1)
    axs.add_artist(circ_2)
    
    plt.xlabel('Radius (m)')
    plt.ylabel('Elevation (m)')
    plt.suptitle(filename)
    plt.title(f'Time = {times[idx]-times[0]:.2f} ms')

writervideo = animation.FFMpegWriter(fps=15) 
ani = animation.FuncAnimation(fig, animate, frames=len(centers_array))
ani.save('animation.mp4', writer=writervideo)
plt.close()

In [ ]:
# save arrays to file
